In [22]:
import numpy as np
import pandas as pd

In [23]:
temp_df = pd.read_csv("/content/IMDB Dataset.csv")

In [24]:
df = temp_df.iloc[:]

In [25]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [26]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [27]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [28]:
df.isnull().sum()

,0
review,0
sentiment,0


In [29]:
df.duplicated().sum()

418

In [30]:
df.drop_duplicates(inplace= True)

In [31]:
df.duplicated().sum()

0

# Basic Preprocessing
- remove tags
- lowercase
- remove stopwords


In [32]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [33]:
df['review']=df['review'].apply(remove_tags)

In [34]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [35]:
df['review']=df['review'].apply(lambda x:x.lower())

In [36]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [39]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
49995,thought movie right good job. creative origina...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,catholic taught parochial elementary schools n...,negative
49998,going disagree previous comment side maltin on...,negative


In [41]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [42]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
49995,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [43]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [44]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [45]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [46]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [47]:
X_train.shape

(39665, 1)

In [48]:
X_test.shape

(9917, 1)

In [49]:
## Applying Bag-of- Word
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
cv=CountVectorizer()

In [51]:
X_train_bow=cv.fit_transform(X_train['review'])
X_test_bow = cv.transform(X_test['review'])

In [52]:
X_train_bow.shape

(39665, 94725)

In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Instantiate the correct model
mnb = MultinomialNB()

# 2. Fit the model on the training data (sparse matrix)
mnb.fit(X_train_bow, y_train)

# 3. Predict on the test data (sparse matrix)
#    Use the correct model variable (mnb, not gnb)
y_pred = mnb.predict(X_test_bow)

# 4. Evaluate the results
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8604416658263587


In [54]:
confusion_matrix(y_test , y_pred)

array([[4418,  615],
       [ 769, 4115]])

In [55]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8550973076535242

In [ ]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8404759503882222

In [ ]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8484420691741454

## Using TF-IDF

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
tfidf = TfidfVectorizer()

In [58]:
X_train_tfidf = tfidf.fit_transform(X_train['review'])
X_test_tfidf = tfidf.transform(X_test['review'])

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8565090249067259

In [2]:
!pip uninstall gensim -y
!pip install gensim

  Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followin

In [1]:
import gensim

In [2]:
from gensim.models import Word2Vec,KeyedVectors

In [3]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
model['cricket'].shape

(300,)

In [59]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
sw_list

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [82]:
# Remove stopwords

X_train = X_train.apply(lambda x: " ".join([item for item in x.split() if item not in sw_list]))
# Remove stopwords

X_test = X_test.apply(lambda x: " ".join([item for item in x.split() if item not in sw_list]))

AttributeError: 'Series' object has no attribute 'split'

In [64]:
import spacy
import en_core_web_sm
# Load the spacy model. This takes a few seconds.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc = nlp(X_train.values[0][0])
print(doc.vector)

[-0.07658163 -0.24747933 -0.07773086 -0.2137088   0.23473561  0.36768228
  0.2356341   0.17590831  0.17741558 -0.23376457  0.34718552 -0.16973095
 -0.19527654  0.05524547 -0.08248196  0.06544644 -0.10968677 -0.19631757
  0.24840833 -0.28058356 -0.19687958  0.2649363  -0.02512992  0.01407745
  0.04850414  0.13255057  0.0350257   0.224932    0.20403923  0.07792556
  0.19378477 -0.02237844  0.02428886  0.14952062 -0.22390169 -0.03031573
  0.02671674  0.11913817 -0.22959958 -0.2529875  -0.06507057  0.04408425
 -0.15551952 -0.01259566  0.1128621   0.10641889  0.05253846  0.0443591
  0.3633255  -0.11093268  0.09391288  0.04384717  0.00687724 -0.05560216
  0.03640988 -0.16977924  0.11866092 -0.09178006  0.15579136  0.06151526
  0.08411172 -0.01218389 -0.02803145 -0.2499171   0.03948826  0.1669046
  0.04710527 -0.23932374 -0.13827941 -0.11487141  0.06410983  0.06758217
 -0.11231832 -0.29035562 -0.06920568 -0.08026807 -0.17250814 -0.0556886
  0.00487634 -0.3065556  -0.2006291  -0.03475934 -0.31

In [65]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz

  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz
ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz for URL https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz


In [66]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [68]:
input_arr = []
for item in X_train.values:
    doc = nlp(item[0])
    input_arr.append(doc.vector)

In [69]:
input_arr = np.array(input_arr)

In [70]:
input_arr.shape

(39665, 96)

In [72]:
input_test_arr = []
for item in X_test.values:
    doc = nlp(item[0])
    input_test_arr.append(doc.vector)

In [73]:
input_test_arr = np.array(input_test_arr)

In [74]:
input_test_arr.shape

(9917, 96)

In [75]:
from sklearn.naive_bayes import GaussianNB

In [76]:
gnb = GaussianNB()
gnb.fit(input_arr,y_train)

GaussianNB()

In [77]:
y_pred = gnb.predict(input_test_arr)
accuracy_score(y_test,y_pred)

0.6085509730765353